## Installation


In [ ]:
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20
!pip install chromadb==0.5.0
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully unins

## Importing Python Modules

In [ ]:
import openai
import numpy as np
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding
import faiss
import warnings
import os
warnings.filterwarnings("ignore")

## Adding OpenAI API key

In [ ]:
from google.colab import userdata

openai.api_key = userdata.get('OPENAI_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_KEY')

## Prompting

RouteGuru is a delivery route assistant designed to optimize the routes for parcel drivers based on road conditions, delivery windows, and customer preferences. Users simply upload a list of parcels to be delivered, and the app analyzes the data to prioritize parcels based on urgency, proximity, and the best available routes.

In [ ]:
system_prompt = """
**Persona Name:** RouteGuru
**Role:** Delivery Route Optimization Assistant
**Personality:** Witty, Expert, Helpful, Adaptive

---

### **1. Role**
**RouteGuru** is an AI-powered delivery route assistant designed to optimize the routes for parcel drivers based on real-time traffic data, road conditions, delivery windows, and customer preferences. RouteGuru’s mission is to help drivers save time, reduce fuel consumption, and ensure timely deliveries.

---

### **2. Instruction**
- **Goal**: Given a list of deliveries, RouteGuru should determine the most **efficient route** based on various factors like traffic, road closures, delivery urgency, and customer preferences.
- **Main Tasks**:
  - **Retrieve** real-time data on traffic, weather, and road closures.
  - **Integrate** the list of deliveries, including specific delivery windows and priority instructions.
  - **Compute** the optimal route based on retrieved data and delivery constraints.
  - **Communicate** the route to the driver, providing clear directions and real-time updates as conditions change.
  - **Execute** by dynamically adjusting the route if there are traffic delays or unforeseen events.

---

### **3. Context**
- **User Base**: The user is a parcel delivery driver who needs to efficiently complete multiple deliveries within a set time frame. The driver may be working for a logistics company, handling a fleet of vehicles, or managing independent deliveries.
- **Environment**: The system is used in a dynamic environment with **real-time traffic**, changing road conditions, and fluctuating delivery constraints.
- **Use Case**: A delivery driver uploads a list of deliveries with addresses, preferred delivery windows, and special instructions (e.g., urgent packages). RouteGuru retrieves relevant external data (e.g., traffic, weather) and computes the best route to follow.
- **Expected Outcomes**: The driver should receive:
  - An optimized delivery route with the fastest possible delivery times.
  - Suggestions for avoiding traffic, road closures, and other delays.
  - Real-time updates on route changes (e.g., accidents, road closures) during the drive.

---

### **4. Constraints**
- **Data Availability**: The system relies on external data sources (e.g., traffic data, road closures, weather updates). If any data is unavailable (e.g., API downtime), the system should either use default assumptions or inform the driver of the limitation.
- **Delivery Priority**: Some deliveries may be time-sensitive or have specific delivery windows that must be met. These should always be prioritized in the route computation.
- **Vehicle Type and Constraints**: RouteGuru must consider the type of vehicle (e.g., delivery van, motorcycle) and any associated constraints, such as size limitations, fuel consumption, or range.
- **Accuracy**: The route suggestions should be **accurate and reliable**, with minimal errors in directions. Traffic information should be **up-to-date** to avoid giving outdated or incorrect routes.
- **Real-Time Updates**: RouteGuru must continuously adjust the route in real-time based on new data about road closures, accidents, or traffic jams. It should ensure the driver avoids delays as much as possible.

---

### **5. Examples**

#### **Example 1: Standard Route Calculation**

**Driver Input**:
"I have 3 deliveries today. Can you calculate the best route?"

**RouteGuru Response**:
"Got it! Let me fetch the latest traffic data and your delivery details...
Here's the optimized route:
1. First, head to Customer A at 123 Main Street. It's the priority delivery and needs to be there by 11 AM.
2. Then, go to Customer B at 456 Oak Road. It’s about 15 minutes away, no traffic expected.
3. Finally, make your way to Customer C at 789 Pine Lane. There’s light traffic around there, but you’ll be done by noon.
Good to go! I'll update you if anything changes."

---

#### **Example 2: Delivery Priority Adjustment**

**Driver Input**:
"I need to make a time-sensitive delivery to Customer A. Can you ensure it’s prioritized?"

**RouteGuru Response**:
"Absolutely. I’ve bumped up Customer A to the top of your route. They’ll get their delivery by 9:30 AM, and I’ll make sure the rest of your deliveries follow in the most efficient order. Let’s get you moving!"
"""

In [ ]:
struct = [{"role": "system", "content": system_prompt}]

## Loading the Dataset

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/noelabu/AIRepublic_Day4/refs/heads/main/dataset/delivery_data_sample.csv', nrows=20)

In [ ]:
df.head(5)

,,parcel_id,customer_name,area,neighborhood,address,nearest_landmark,latitude,longitude,parcel_weight,special_instructions,...,road_restrictions,alternative_routes_available,delivery_time_window_start,delivery_time_window_end,weather_impact,delivery_priority,expected_delivery_time,expected_delivery_duration,route_feedback,additional_notes
P001,John Doe,Quezon City,Tomas Morato,45A,12th St,QC,GMA Network,14.6451,121.0362,3.5 kg,Leave at front door,...,Heavy traffic at 7 AM,Yes,08:00,10:00,Rain expected,High,09:00,30 minutes,Positive,Customer prefers fast delivery
P002,Maria Cruz,Makati,Legaspi Village,215 Greenbelt Building,Greenbelt Mall,14.5592,121.0223,1.2 kg,Leave with security,Office delivery,No parking available,...,12:00,14:00,NaN,Medium,13:00,20 minutes,Neutral,Must be delivered before lunch hour,NaN,NaN
P003,James Smith,Pasig,Ortigas,42 F. Ortigas Ave.,Robinsons Galleria,14.5846,121.0589,5.0 kg,Leave at front desk,Pick up at location,Construction delays,...,10:00,12:00,Cloudy,slight rain,High,11:00,25 minutes,Negative,Suggest alternative route to avoid construction,NaN
P004,Ellen Lee,Quezon City,Novaliches,31,Sampaguita St.,SM City North EDSA,14.7589,121.0394,7.5 kg,Call upon arrival,Home delivery,...,Yes,16:00,18:00,Clear skies,Low,17:30,40 minutes,Positive,No specific requirements,NaN
P005,Karen Garcia,Makati,Bel-Air,15 Juan Luna St.,Rockwell Mall,14.5538,121.0209,4.0 kg,Leave with concierge,Office delivery,No access from main road,...,14:00,16:00,NaN,Medium,15:00,30 minutes,Neutral,Additional stop for drop-off required,NaN,NaN


In [ ]:
df["combined"] = df.apply(lambda x: ' '.join(x.values.astype(str)), axis=1)

In [ ]:
documents = df['combined'].tolist()

In [ ]:
embeddings = [get_embedding(doc, engine = "text-embedding-3-small") for doc in documents]

In [ ]:
embeddings_dim = len(embeddings[0])

In [ ]:
embeddings_np = np.array(embeddings).astype('float32')

In [ ]:
index = faiss.IndexFlatL2(embeddings_dim)

In [ ]:
index.add(embeddings_np)

In [ ]:
user_message = "Is it good to deliver the parcel to SM Aura first?"

In [ ]:
query_embedding = get_embedding(user_message, engine='text-embedding-3-small')
query_embedding_np = np.array([query_embedding]).astype('float32')

In [ ]:
_, indices = index.search(query_embedding_np, 5)

In [ ]:
retrieved_docs = [documents[i] for i in indices[0]]
retrieved_docs

['Taguig Bonifacio Global City 8th Avenue  BGC SM Aura 14.5500 121.0500 2.3 kg Leave at front door Home delivery No heavy vehicles allowed on 8th Avenue Yes 09:00 11:00 Sunny  no rain expected High 10:00 15 minutes Positive Fast delivery requested',
 'Pasay Mall of Asia 2F SM Mall of Asia MOA Arena 14.5509 120.9833 6.0 kg Leave with security guard Office delivery High congestion near SM Mall of Asia No 15:00 17:00 Cloudy  mild rain Medium 16:00 35 minutes Neutral Keep package protected due to rain nan',
 'Mandaluyong Mandaluyong City 35 EDSA  Ortigas Center SM Megamall 14.5853 121.0586 8.2 kg Leave with security guard Office delivery EDSA traffic delays Yes 10:00 12:00 Clear skies High 11:30 45 minutes Positive Delivery should be quick due to office hours nan',
 'Pasig Kapitolyo 67 Shaw Blvd. SM Megamall 14.5840 121.0568 6.0 kg Leave at front desk Pick up at location No heavy vehicles allowed Yes 08:30 10:00 Clear skies Low 09:15 20 minutes Positive No issues  easy delivery nan',
 'Mak

In [ ]:
context = ' '.join(retrieved_docs)

In [ ]:
structured_prompt = f"Context:\n{context}\n\nQuery:\n{user_message}\n\nResponse:"

In [ ]:
print(structured_prompt)

Context:
Taguig Bonifacio Global City 8th Avenue  BGC SM Aura 14.5500 121.0500 2.3 kg Leave at front door Home delivery No heavy vehicles allowed on 8th Avenue Yes 09:00 11:00 Sunny  no rain expected High 10:00 15 minutes Positive Fast delivery requested Pasay Mall of Asia 2F SM Mall of Asia MOA Arena 14.5509 120.9833 6.0 kg Leave with security guard Office delivery High congestion near SM Mall of Asia No 15:00 17:00 Cloudy  mild rain Medium 16:00 35 minutes Neutral Keep package protected due to rain nan Mandaluyong Mandaluyong City 35 EDSA  Ortigas Center SM Megamall 14.5853 121.0586 8.2 kg Leave with security guard Office delivery EDSA traffic delays Yes 10:00 12:00 Clear skies High 11:30 45 minutes Positive Delivery should be quick due to office hours nan Pasig Kapitolyo 67 Shaw Blvd. SM Megamall 14.5840 121.0568 6.0 kg Leave at front desk Pick up at location No heavy vehicles allowed Yes 08:30 10:00 Clear skies Low 09:15 20 minutes Positive No issues  easy delivery nan Makati Legas

In [ ]:
print(user_message)

Is it good to deliver the parcel to SM Aura first?


In [ ]:
chat =  openai.ChatCompletion.create(model = "gpt-4o-mini", messages = struct + [{"role": "user", "content" : structured_prompt}], temperature=0.5, max_tokens=1500, top_p=1, frequency_penalty=0, presence_penalty=0)
struct.append({"role": "user", "content": user_message})
response = chat.choices[0].message.content
struct.append({"role": "assistant", "content": response})

In [ ]:
print(response)

"Delivering to SM Aura first is not the best option. Here’s why:

### Delivery Schedule Review:
1. **SM Aura**  
   - **Delivery Window**: 9:00 AM to 11:00 AM  
   - **Traffic Conditions**: No heavy vehicles allowed on 8th Avenue  
   - **Weight**: 2.3 kg  
   - **Notes**: Fast delivery requested.

2. **SM Megamall**  
   - **Delivery Window**: 10:00 AM to 12:00 PM  
   - **Traffic Conditions**: EDSA traffic delays  
   - **Weight**: 8.2 kg  
   - **Notes**: Delivery should be quick due to office hours.

3. **Mall of Asia**  
   - **Delivery Window**: 3:00 PM to 5:00 PM  
   - **Traffic Conditions**: High congestion near SM Mall of Asia  
   - **Weight**: 6.0 kg  
   - **Notes**: Keep the package protected due to rain.

4. **SM Megamall (Pickup)**  
   - **Delivery Window**: 8:30 AM to 10:00 AM  
   - **Traffic Conditions**: No heavy vehicles allowed  
   - **Weight**: 6.0 kg  
   - **Notes**: Easy delivery.

5. **Greenbelt Mall**  
   - **Delivery Window**: 12:00 PM to 2:00 PM  
   - 